In [ ]:
# IPython magig  tools
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path

from aind_vr_foraging_analysis.utils import parse, processing, plotting_utils as plotting, AddExtraColumns
from aind_vr_foraging_analysis.utils import breathing_signal as breathing

# Plotting libraries
import matplotlib.pyplot as plt


import seaborn as sns
import pandas as pd
import numpy as np
import datetime

sns.set_context('talk')

import warnings
pd.options.mode.chained_assignment = None  # Ignore SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

color1='#d95f02'
color2='#1b9e77'
color3='#7570b3'
color4='yellow'
odor_list_color = [color1, color2, color3, color4]

pdf_path = r'Z:\scratch\vr-foraging\sessions'
base_path = 'Z:/scratch/vr-foraging/data/'
foraging_figures = r'C:\Users\tiffany.ona\OneDrive - Allen Institute\Documents'

from scipy.optimize import curve_fit


In [ ]:
def parse_session(mouse, date):
    session_found = False
    directory = os.path.join(base_path, mouse)
    files = os.listdir(os.path.join(base_path, mouse))

    sorted_files = sorted(files, key=lambda x: os.path.getctime(os.path.join(directory, x)), reverse=True)

    # All this segment is to find the correct session without having the specific path
    for file_name in sorted_files:
        
        if session_found == True:
            break
        
        print(file_name)
        # Find specific session sorted by date
        session = file_name[-15:-7]
        if datetime.datetime.strptime(session, "%Y%m%d").date() != date:
            continue
        else:
            print('correct date found')
            session_found = True
            
        # Recover data streams
        session_path = os.path.join(base_path, mouse, file_name)
        session_path = Path(session_path)
        data = parse.load_session_data(session_path)
        
        # Load the encoder data separately
        stream_data = parse.ContinuousData(data)
        
        return stream_data

In [ ]:
# Non-shielded cable
date = datetime.date.today()
date_string = "10/24/2024"
date = datetime.datetime.strptime(date_string, "%m/%d/%Y").date()
mouse = '745305'
stream_data_before = parse_session(mouse, date)

# Shielded cable
date = datetime.date.today()
date_string = "10/28/2024"
date = datetime.datetime.strptime(date_string, "%m/%d/%Y").date()
mouse = '745305'
stream_data_after = parse_session(mouse, date)

In [ ]:
signal = stream_data_before.breathing.data.values
zero_index = stream_data_before.breathing.index[0]

fig, axs = plt.subplots(2,2, figsize=(12,8), sharey=True)
plt.suptitle(f'{mouse}')

i=0
for ax in axs.flatten()[:2]:
    ax.plot(stream_data_before.breathing.index-zero_index, signal, color= 'black')

    ax.set_ylim(1800, 2600)
    ax.set_xlim(0+i, 5+i)
    ax.set_ylabel('Breathing (a.u.)')
    ax.set_xlabel('Time (s) with non-shielded cable')
    i+=10

signal = stream_data_after.breathing.data.values
zero_index = stream_data_after.breathing.index[0]

for ax in axs.flatten()[2:]:
    ax.plot(stream_data_after.breathing.index-zero_index, signal, color= 'black')

    ax.set_ylim(1800, 2600)
    ax.set_xlim(0+i, 5+i)
    ax.set_ylabel('Breathing (a.u.)')
    ax.set_xlabel('Time (s) with shielded cable')
    i+=10

sns.despine()
plt.tight_layout()